In [ ]:
import hsfs
import pandas as pd

In [ ]:
connection = hsfs.connection()
fs = connection.get_feature_store()

In [ ]:
pdf = pd.read_csv("hdfs:///Projects/live_coding/RawData/transactions.csv", parse_dates=["datetime"])

In [ ]:
pdf.sort_values("datetime", inplace=True)

In [ ]:
pdf = pdf[['datetime', 'amount', 'cc_num']]

In [ ]:
roll_group_by = pdf.groupby("cc_num").rolling("10min", on="datetime")

In [ ]:
roll_group_by.count()

In [ ]:
df_10_count = pd.DataFrame(roll_group_by.count())
df_10_count.reset_index(level=["cc_num"], inplace=True)
df_10_count.sort_index(inplace=True)
df_10_count.rename(columns={"amount": "num_trans_per_10m"}, inplace=True)

In [ ]:
df_10_avg = pd.DataFrame(roll_group_by.mean())
df_10_avg.reset_index(level=["cc_num"], inplace=True)
df_10_avg.sort_index(inplace=True)
df_10_avg.rename(columns={"amount": "avg_amt_per_10m"}, inplace=True)

In [ ]:
df_10_std = pd.DataFrame(roll_group_by.std())
df_10_std.reset_index(level=["cc_num"], inplace=True)
df_10_std.sort_index(inplace=True)
df_10_std.rename(columns={"amount": "stdev_amt_per_10m"}, inplace=True)

In [ ]:
df_10_std

In [ ]:
mrg = df_10_count.merge(df_10_avg)

In [ ]:
final_df = mrg.merge(df_10_std, on=['cc_num', 'datetime'])

In [ ]:
final_df['datetime'] = pd.Timestamp.now()

In [ ]:
final_df

In [ ]:
profiles_activity_10m = fs.get_or_create_feature_group(
    name="profiles_activity_10m",
    version=1,
    description="Credit card activity over 10 minutes window",
    primary_key=['cc_num'],
    event_time="datetime",
    online_enabled=True,
    statistics_config={'histograms': True, 'correlations': True},
    stream=True
)

In [ ]:
profiles_activity_10m.insert(final_df)